In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib

# Get domain parameters
def getInputs():
    L = float(input("Enter domain length (m): "))
    T = float(input("Enter total simulation time (s): "))
    U = float(input("Enter river velocity U (m/s): "))
    k = 0  # no decay for Test Case 2

    print("\nGrid setup options:")
    print("1. Input number of grid points (Nx, Nt)")
    print("2. Input step sizes (Δx, Δt)")
    choice = input("Enter 1 or 2: ")

    if choice == "1":
        Nx = int(input("Enter Nx (number of spatial points): "))
        Nt = int(input("Enter Nt (number of time steps): "))
        dx = L / (Nx - 1)
        dt = T / (Nt - 1)
    else:
        dx = float(input("Enter Δx (spatial step size in m): "))
        dt = float(input("Enter Δt (time step size in s): "))
        Nx = int(L / dx) + 1
        Nt = int(T / dt) + 1

    return L, T, U, Nx, Nt, dx, dt, k

def read_initial_conditions(filename, model_x):
    """
    Reads initial conditions from a CSV file and interpolates them
    onto the model grid.
    """
    my_data = pd.read_csv(filename, header=0, index_col=0, encoding='latin-1')
    measurement_x = my_data.index.values.astype(float)   # x positions from CSV
    measurement_values = my_data.iloc[:, 0].values       # concentration values

    # Interpolate onto model grid
    theta_init = np.interp(model_x[:-1], measurement_x, measurement_values)
    return theta_init

def concovertime(L, T, U, Nx, Nt, dx, dt, decay=0, init_file=None):
    # Create grids
    x = np.linspace(0, L, Nx)
    theta_new = np.zeros(Nx-1)
    theta_old = np.zeros(Nx-1)

    # Arrays for matrix coefficients
    A = np.zeros(Nx-1)
    B = np.zeros(Nx-1)
    F = np.zeros(Nx-1)

    history = np.zeros((Nt, Nx-1))

    # Constant velocity
    u = np.full(Nx-1, U)

    # Read and interpolate initial conditions if provided
    if init_file:
        theta_old[:] = read_initial_conditions(init_file, x)
    else:
        theta_old[0] = 250.0  # default boundary

    history[0, :] = theta_old.copy()

    # Main simulation loop
    for j in range(1, Nt):
        theta_new[:] = 0.0
        theta_new[0] = theta_old[0]  # maintain boundary

        # Store values in A and B
        for i in range(Nx-1):
            A[i] = (1.0 / dt) + (u[i] / dx)
            B[i] = - u[i] / dx

        # RHS vector F
        for i in range(Nx-2):
            F[i] = (1/dt) * theta_old[i+1]

        # Forward substitution
        for I in range(1, Nx-1):
            theta_new[I] = (1/A[I-1]) * (F[I-1] - B[I-1]*theta_new[I-1])

        history[j] = theta_new.copy()
        theta_old[:] = theta_new[:]

    print("A[0:3] =", A[:3])
    print("B[0:3] =", B[:3])
    print("theta_new[0:10] =", theta_new[:10])

    # Animation
    matplotlib.rcParams["animation.html"] = "jshtml"
    matplotlib.rcParams['figure.dpi'] = 150
    plt.ioff()
    fig, ax = plt.subplots(figsize=(8,5))

    def animate(frame):
        ax.cla()
        ax.plot(x[:-1], history[frame, :], color='blue')
        ax.set_xlabel("Distance downstream (m)")
        ax.set_ylabel("Concentration (µg/m³)")
        ax.set_ylim(0, max(history.max()*1.1, 260))
        ax.grid(True)
        ax.set_title(f"Pollutant concentration at t = {frame*dt:.1f} s")

    ani = FuncAnimation(fig, animate, frames=Nt, interval=50)
    return ani

# Run test case 2
L, T, U, Nx, Nt, dx, dt, k = getInputs()
concovertime(L, T, U, Nx, Nt, dx, dt, k, init_file='initial_conditions.csv')


Enter domain length (m): 20
Enter total simulation time (s): 300
Enter river velocity U (m/s): 0.1

Grid setup options:
1. Input number of grid points (Nx, Nt)
2. Input step sizes (Δx, Δt)
Enter 1 or 2: 2
Enter Δx (spatial step size in m): 0.2
Enter Δt (time step size in s): 10
A[0:3] = [0.6 0.6 0.6]
B[0:3] = [-0.5 -0.5 -0.5]
theta_new[0:10] = [300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
